<a href="https://colab.research.google.com/github/rahiakela/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/blob/18-reinforcement-learning/2_introduction_to_markov_decision_processes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Markov Decision Processes

In the early 20th century, **the mathematician Andrey Markov studied stochastic processes with no memory, called Markov chains. Such a process has a fixed number of states, and it randomly evolves from one state to another at each step. The probability for it to evolve from a state s to a state s′ is fixed, and it depends only on the pair (s, s′), not on past states (this is why we say that the system has no memory)**.

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/markov-chain.png?raw=1' width='800'/>

Suppose that the process starts in state $s_0$, and there is a 70% chance that it will remain in that state at the next step. Eventually it is bound to leave that state and never come back because no other state points back to $s_0$. If it goes to state $s_1$, it will then most likely go to state $s_2$ (90% probability), then immediately back to state $s_1$ (with 100% probability). It may alternate a number of times between these two states, but eventually it will fall into state $s_3$ and remain there forever (this is a terminal
state). Markov chains can have very different dynamics, and they are heavily used in thermodynamics, chemistry, statistics, and much more.



## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Markov Chains

In [2]:
np.random.seed(42)

# shape=[s, s']
transition_probabilities = [
    [0.7, 0.2, 0.0, 0.1],  # from s0 to s0, s1, s2, s3
    [0.0, 0.0, 0.9, 0.1],  # from s1 to ...
    [0.0, 1.0, 0.0, 0.0],  # from s2 to ...
    [0.0, 0.0, 0.0, 1.0]   # from s3 to ...                       
]

n_max_steps = 50

def print_sequence():
  current_state = 0
  print('States:', end=' ')
  for step in range(n_max_steps):
    print(current_state, end=' ')
    if current_state == 3:
      break
    current_state = np.random.choice(range(4), p=transition_probabilities[current_state])
  else:
    print('...', end='')
  print()

for _ in range(10):
  print_sequence()

States: 0 0 3 
States: 0 1 2 1 2 1 2 1 2 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 ...
States: 0 0 3 
States: 0 0 0 1 2 1 2 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 


## Markov Decision Process

Markov decision processes were first [described in the 1950s by Richard Bellman](https://apps.dtic.mil/dtic/tr/fulltext/u2/606367.pdf).
**They resemble Markov chains but with a twist: at each step, an agent can choose one of several possible actions, and the transition probabilities depend on the chosen action.** Moreover, some state transitions return some reward (positive or negative), and the agent’s goal is to find a policy that will maximize reward over time.

For example, the MDP has three states (represented by circles) and up to three possible discrete actions at each step (represented by diamonds).

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/markov-decision-process.png?raw=1' width='800'/>

If it starts in state $s_0$, the agent can choose between actions $a_0, a_1$, or $a_2$. If it chooses action $a_1$, it just remains in state $s_0$ with certainty, and without any reward. It can thus decide to stay there forever if it wants to. But if it chooses action $a_0$, it has a 70% probability
of gaining a reward of +10 and remaining in state $s_0$. 

It can then try again and again to gain as much reward as possible, but at one point it is going to end up instead in state $s_1$. In state $s_1$ it has only two possible actions: $a_0$ or $a_2$. It can choose to stay put by repeatedly choosing action $a_0$, or it can choose to move on to state $s_2$ and get a negative reward of –50 (ouch). In state $s_2$ it has no other choice than to take action $a_1$, which will most likely lead it back to state $s_0$, gaining a reward of +40 on the way. 

You get the picture. By looking at this MDP, can you guess which strategy will
gain the most reward over time? In state $s_0$ it is clear that action $a_0$ is the best option, and in state $s_2$ the agent has no choice but to take action $a_1$, but in state $s_1$ it is not obvious whether the agent should stay put ($a_0$) or go through the fire ($a_2$).

Bellman found a way to estimate the optimal state value of any state $s$, noted $V*(s)$, which is the sum of all discounted future rewards the agent can expect on average after it reaches a state s, assuming it acts optimally.

He showed that if the agent acts optimally, then the Bellman Optimality Equation applies.This recursive equation says that if the agent acts optimally, then the optimal value of the current state is equal to the reward it will get on average after taking one optimal action, plus the expected optimal value of all possible next states that this action can lead to.

$$V^*(s) = max_a Σ_sT(s,a,s′)[R(s,a,s′) + γ · V*(s′)]$$

In this equation:

* $T(s, a, s′)$ is the transition probability from state $s$ to state $s′$, given that the agent chose action $a$. For example,$T(s_2, a_1, s_0) = 0.8$.

* $R(s, a, s′)$ is the reward that the agent gets when it goes from state $s$ to state $s′$, given that the agent chose action $a$. For example, $R(s2, a1,
s0) = +40.$

* $γ$ is the discount factor.

This equation leads directly to an algorithm that can precisely estimate the optimal state value of every possible state: **you first initialize all the state value estimates to zero, and then you iteratively update them using the Value Iteration algorithm.**A remarkable result is that, given enough time, these estimates are guaranteed to converge to the optimal state values, corresponding to the optimal policy.

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/value-iteration-algorithm.png?raw=1' width='800'/>

In this equation, $V_k(s)$ is the estimated value of state $s$ at the $k^{th}$ iteration of the algorithm.

> This algorithm is an example of Dynamic Programming, which
breaks down a complex problem into tractable subproblems that
can be tackled iteratively.

Knowing the optimal state values can be useful, in particular to evaluate a policy, but it does not give us the optimal policy for the agent. Luckily, Bellman found a very similar algorithm to estimate the optimal state-action values, generally called QValues (Quality Values). The optimal Q-Value of the state-action pair $(s, a)$, noted $Q*(s, a)$, is the sum of discounted future rewards the agent can expect on average after it reaches the state s and chooses action $a$, but before it sees the outcome of this action, assuming it acts optimally after that action.

Here is how it works: **once again, you start by initializing all the Q-Value estimates to zero, then you update them using the Q-Value Iteration algorithm.**

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/q-value-iteration-algorithm.png?raw=1' width='800'/>

Once you have the optimal Q-Values, defining the optimal policy, noted $π*(s)$, is trivial: when the agent is in state $s$, it should choose the action with the highest Q-Value for that state: $π^*(s) = argmax Q^*(s, a)$.

Let’s apply this algorithm to the MDP. First, we need to
define the MDP:


In [0]:
# shape=[s, a, s']
transition_probabilities = [
   [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
   [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
   [None, [0.8, 0.1, 0.1], None]                         
]

# shape=[s, a, s']
rewards = [
   [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
   [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
   [[0, 0, 0], [+40, 0, 0], [0, 0, 0]]        
]

possible_actions = [
    [0, 1, 2], [0, 2], [1]                
]

For example, to know the transition probability from $s_2$ to $s_0$ after playing action a1, we will look up transition_probabilities[2][1][0] (which is 0.8). Similarly, to get the corresponding reward, we will look up rewards[2][1][0] (which is +40). And to get the list of possible actions in $s_2$, we will look up possible_actions[2] (in this case, only action $a_1$ is possible).

Next, we must initialize all the Q-Values to 0 (except for the the impossible actions, for which we set the Q-Values to –∞):

In [0]:
Q_values = np.full((3, 3), -np.inf)  # -np.inf for impossible actions
for state, actions in enumerate(possible_actions):
  Q_values[state, actions] = 0.0    # for all possible actions

Now let’s run the Q-Value Iteration algorithm. It applies repeatedly, to all Q-Values, for every state and every possible action:

In [0]:
# the discount factor
gamma = 0.90

history1 = []
for iteration in range(50):
  Q_prev = Q_values.copy()
  history1.append(Q_prev)
  for s in range(3):
    for a in possible_actions[s]:
      Q_values[s, a] = np.sum([transition_probabilities[s][a][sp] * (rewards[s][a][sp] + gamma * np.max(Q_prev[sp])) for sp in range(3)])
history1 = np.array(history1)

That’s it! The resulting Q-Values look like this:

In [6]:
Q_values

array([[18.91891892, 17.02702702, 13.62162162],
       [ 0.        ,        -inf, -4.87971488],
       [       -inf, 50.13365013,        -inf]])

For example, when the agent is in state $s_0$ and it chooses action $a_1$, the expected sum of discounted future rewards is approximately 17.0.

For each state, let’s look at the action that has the highest Q-Value:

In [7]:
np.argmax(Q_values, axis=1)  # optimal action for each state

array([0, 0, 1])

This gives us the optimal policy for this MDP, when using a discount factor of 0.90: in state $s_0$ choose action $a_0$; in state $s_1$ choose action $a_0$ (i.e., stay put); and in state $s_2$ choose action $a_1$ (the only possible action).

Interestingly, if we increase the discount factor to 0.95, the optimal policy changes: in state $s_1$ the best action becomes $a_2$ (go through the fire!). This makes sense because the more you value future rewards, the more you are willing to put up with some pain now for the promise of future bliss.

Let's try again with a discount factor of 0.95:

In [0]:
Q_values = np.full((3, 3), -np.inf)  # -np.inf for impossible actions
for state, actions in enumerate(possible_actions):
  Q_values[state, actions] = 0.0    # for all possible actions

In [0]:
# the discount factor
gamma = 0.95

for iteration in range(50):
  Q_prev = Q_values.copy()
  for s in range(3):
    for a in possible_actions[s]:
      Q_values[s, a] = np.sum([transition_probabilities[s][a][sp] * (rewards[s][a][sp] + gamma * np.max(Q_prev[sp])) for sp in range(3)])

In [10]:
Q_values

array([[21.73304188, 20.63807938, 16.70138772],
       [ 0.95462106,        -inf,  1.01361207],
       [       -inf, 53.70728682,        -inf]])

In [11]:
np.argmax(Q_values, axis=1)  # optimal action for each state

array([0, 2, 1])

Now the policy has changed! In state $s_1$, we now prefer to go through the fire (choose action $a_2$). This is because the discount factor is larger so the agent values the future more, and it is therefore ready to pay an immediate penalty in order to get more future rewards.

## Temporal Difference Learning